This notebook contains the code blocks of [this](opengm_quickstart.md) document. 

In [1]:
import opengm
gm = opengm.gm([2, 2, 2, 2], operator='multiplier')

In [2]:
import numpy 
f0_params = numpy.array([0.3, 0.7])
f1_params = numpy.array([[0.2, 0.8],
                         [0.7, 0.3]])
f2_params = numpy.array([[0.4, 0.6],
                         [0.3, 0.7]])
f3_params = numpy.array([
    [[0.2, 0.8],
     [0.3, 0.7]],
    [[0.6, 0.4],
     [0.7, 0.3]]])

In [3]:
gm.addFactor(gm.addFunction(f0_params),[0])
gm.addFactor(gm.addFunction(f1_params),[0, 1])
gm.addFactor(gm.addFunction(f2_params),[0, 2])
gm.addFactor(gm.addFunction(f3_params), [1, 2, 3])

3

In [4]:
gm2, vs = gm.fixVariables([2], [1])

In [5]:
print(vs)

[0 1 3]


In [6]:
from __future__ import print_function
for f in gm2.factors():
    print(f.variableIndices, f.copyValues())

[0, ] [ 0.3  0.7]
[0, 1, ] [ 0.2  0.7  0.8  0.3]
[0, ] [ 0.6  0.7]
[1, 2, ] [ 0.3  0.7  0.7  0.3]
[] [ 1.]


In [7]:
gm3, vs3 = gm.fixVariables([3], [1])

inference_parameters = opengm.InfParam(
    steps=10, 
    damping=0.5,
    convergenceBound=0.001)

inference_algorithm = opengm.inference.BeliefPropagation(
    gm3,
    accumulator='integrator',
    parameter=inference_parameters)

inference_algorithm.infer()

for i in vs3:
    print(inference_algorithm.marginals(i))

[[ 0.22971516  0.77028484]]
[[ 0.72716989  0.27283011]]
[[ 0.36386238  0.63613762]]


In [8]:
class PyCallback(object):
    def __init__(self):
        pass
    def begin(self, inference):
        print("begin")
    def end(self, inference):
        print("end")
    def visit(self, inference):
        print("marginals of variable 2: ", inference.marginals(2))

callback=PyCallback()

inference_algorithm = opengm.inference.BeliefPropagation(
    gm3,
    accumulator='integrator',
    parameter=inference_parameters)

visitor=inference_algorithm.pythonVisitor(callback,visitNth=1)

inference_algorithm.infer(visitor)

begin
marginals of variable 2:  [[ 0.43992727  0.56007273]]
marginals of variable 2:  [[ 0.40775119  0.59224881]]
marginals of variable 2:  [[ 0.39028824  0.60971176]]
marginals of variable 2:  [[ 0.3803642  0.6196358]]
marginals of variable 2:  [[ 0.37435229  0.62564771]]
marginals of variable 2:  [[ 0.37047283  0.62952717]]
marginals of variable 2:  [[ 0.36784895  0.63215105]]
marginals of variable 2:  [[ 0.36602756  0.63397244]]
marginals of variable 2:  [[ 0.36475245  0.63524755]]
marginals of variable 2:  [[ 0.36386238  0.63613762]]
end


opengm.inference._inference.InferenceTermination.NORMAL